In [1]:
print 1

1


In [2]:
import sys
sys.path.append('/afs/ee.cooper.edu/user/t/a/tam8/documents/ml_misc/semi_supervised/')
import coreg
reload(coreg)
import rasco
reload(rasco)
import trireg
reload(trireg)
sys.path.append('/afs/ee.cooper.edu/user/t/a/tam8/documents/ml_misc/ordinal/')
import simple
reload(simple)
sys.path.append('/afs/ee.cooper.edu/user/t/a/tam8/documents/ml_misc/ensemble/')
import stacking
reload(stacking)

sys.path.append('/afs/ee.cooper.edu/user/t/a/tam8/documents/ml_misc/neighbors/')
import rpfnn
reload(rpfnn)
import ann
reload(ann)

import pandas as pd
import numpy as np
from sklearn import preprocessing
import xgboost as xgb

import transformers as tforms
reload(tforms)
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion, make_union

import metrics
reload(metrics)
from sklearn.cross_validation import StratifiedKFold, train_test_split

from sklearn.base import clone
from sklearn.svm import SVC, LinearSVC, SVR
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression, LogisticRegression, ElasticNet, Ridge, Lasso, SGDRegressor, SGDClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier, DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.lda import LDA
from sklearn.qda import QDA
from sklearn.neighbors import KNeighborsRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.isotonic import IsotonicRegression
from sklearn.preprocessing import StandardScaler

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout, MaxoutDense, Reshape
from keras.layers.normalization import BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from keras.optimizers import Adadelta, Adagrad, Adam, RMSprop
from keras.layers.advanced_activations import ParametricSoftplus, PReLU

from svmlight_loader import dump_svmlight_file, load_svmlight_file

from collections import Counter
import minirank as mr

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from time import time

import cPickle as pickle

def wgmean(x, w):
    return np.exp(np.sum(w*np.log(x), axis=1) / np.sum(w, axis=1))

import logging 



Using gpu device 0: GeForce GTX 580


In [3]:
# Load Data le instead
# train_pd  = pd.read_pickle('saved/train_pd_l_enc.p')
# test_pd  = pd.read_pickle('saved/test_pd_l_enc.p')

train_pd  = pd.read_pickle('saved/train_pd_le_and_oh_enc.p')
test_pd  = pd.read_pickle('saved/test_pd_le_and_oh_enc.p')

labels = pd.read_pickle('saved/labels.p')
test_ind = pickle.load(open('saved/test_ind.p'))
y_binned = np.load('saved/y_binned.npy')

# drop_cols = ['T1_V10', 'T1_V13', 'T2_V7', 'T2_V10']
drop_cols = []
# Lets not drop anything

# drop_cols = train_pd.columns[fi < 0.01]

for col in drop_cols:
    train_pd.drop(col, axis=1, inplace=True)
    test_pd.drop(col, axis=1, inplace=True)

In [4]:
X_tsne2_26 = np.load('saved/X_tsne2_26important.npy')
X_tsne2_26_train = X_tsne2_26[:len(train_pd), :]
X_tsne2_26_test = X_tsne2_26[-len(test_pd):, :]

X_tsne2 = np.load('saved/X_tsne2.npy')
X_tsne2_train = X_tsne2[:len(train_pd), :]
X_tsne2_test = X_tsne2[-len(test_pd):, :]

print X_tsne2_train.shape
print X_tsne2_test.shape
print X_tsne2_26_train.shape
print X_tsne2_26_test.shape

(50999, 2)
(51000, 2)
(50999, 2)
(51000, 2)


In [5]:
# mapped_train, _y = load_svmlight_file('saved/mapped2000_train.libsvm')
# mapped_test, _y = load_svmlight_file('saved/mapped2000_test.libsvm')

mapped_train, _y = load_svmlight_file('saved/mapped2000_train.libsvm')
mapped_test, _y = load_svmlight_file('saved/mapped2000_test.libsvm')

X_2000mean_train = mapped_train.todense()
X_2000mean_test = mapped_test.todense()
print X_2000mean_train.shape
print X_2000mean_test.shape

(50999, 2000)
(51000, 2000)


In [6]:
train = np.array(train_pd)
test = np.array(test_pd)

X_train = train.astype(float)
X_test = test.astype(float)
y_train = np.array(labels)

try:
    X_train = np.c_[X_train, X_tsne2_26_train]
    X_test = np.c_[X_test, X_tsne2_26_test]
except:
    pass

try:
    X_train = np.c_[X_train, X_2000mean]
except:
    pass

holdout = False
if holdout:
    X_train, X_hold, \
    y_train, y_hold, \
    y_binned, y_binned_hold \
    = train_test_split(
        X_train, y_train, y_binned, 
        test_size=0.2, random_state=0)

# """
pipe_x = make_pipeline(
    make_union(
        tforms.IdentityTformer(),
    ),
#     StandardScaler(),
)
pipe_y = make_pipeline(
    tforms.IdentityTformer(),    
)
pipe_x.fit(np.r_[X_train, X_test])
pipe_y.fit(y_train)

X_train = pipe_x.transform(X_train)
X_test = pipe_x.transform(X_test)
try:
    X_hold = pipe_x.transform(X_hold)
except:
    pass
# y_train = pipe_y.fit_transform(y_train)

# """

print 'y_train', y_train.shape
print 'X_train', X_train.shape
print 'X_test', X_test.shape
try:
    print 'X_hold', X_hold.shape
except:
    pass
print len(np.unique(y_train))
print len(np.unique(y_binned))
print y_binned.shape
print type(X_train[0][0])

y_train (50999,)
X_train (50999, 129)
X_test (51000, 129)
X_hold 50
5
(50999,)
<type 'numpy.float64'>


In [7]:
X_nn_train = np.c_[X_2000mean_train, X_train]
X_nn_test = np.c_[X_2000mean_test, X_test]
print X_nn_train.shape
print X_nn_test.shape

(50999, 2129)
(51000, 2129)


# DEFINE THE MODELS
## BASE MODELS

### NN (using sofia kmeans cluster distance maps)

In [8]:
n_feats = X_nn_train.shape[1]
drop_prob = 0.6

model = Sequential()

hidden_size = 1024
model.add(Dense(n_feats, hidden_size))
model.add(Dropout(0.2))
model.add(ParametricSoftplus(hidden_size))
model.add(BatchNormalization((hidden_size,)))
model.add(Dropout(drop_prob))
model.add(Dense(hidden_size, hidden_size))
model.add(ParametricSoftplus(hidden_size))
model.add(BatchNormalization((hidden_size,)))
model.add(Dropout(drop_prob))
model.add(Dense(hidden_size, hidden_size))
model.add(ParametricSoftplus(hidden_size))
model.add(BatchNormalization((hidden_size,)))
model.add(Dropout(drop_prob))
model.add(Dense(hidden_size, hidden_size))
model.add(ParametricSoftplus(hidden_size))
model.add(BatchNormalization((hidden_size,)))
model.add(Dropout(drop_prob))
model.add(Dense(hidden_size, 1))
model.add(Activation('linear'))

loss_type = 'msle'

opt = RMSprop(lr=0.0005, rho=0.75, epsilon=1e-6)

nn_reg = KerasRegressor(model, optimizer=opt, loss=loss_type,
                        train_batch_size=1024*4, test_batch_size=1024*16,
                        nb_epoch=500)

### GBM XGBOOST

In [9]:
gbm_reg = xgb.XGBRegressor(
    objective="reg:linear",
    n_estimators=900,
    learning_rate=0.005,
#     gamma=0.0,
    max_depth=9,
    min_child_weight=6,
#     max_delta_step=10,
    subsample=0.7,
    colsample_bytree=0.7,
#     scale_pos_weight=1.0,
    base_score=0.5,
    nthread=7,
    seed=322,
    silent=True,
)

### RF

In [10]:
rf_reg = RandomForestRegressor(n_jobs=-1, **{'max_leaf_nodes': None, 
                                         'bootstrap': False, 
                                         'min_samples_leaf': 8, 
                                         'n_estimators': 150, 
                                         'min_samples_split': 8, 
                                         'min_weight_fraction_leaf': 0.25, 
                                         'max_features': 'sqrt', 
                                         'max_depth': 12} )

### Other crappy regressors

In [11]:
lin_reg = LinearRegression()
ridge_reg = Ridge()

### Custom weird stuff

In [12]:
h = [GaussianNB() for _ in range(64)] + \
    [DecisionTreeClassifier(max_depth=6, min_samples_split=5, class_weight='auto') for _ in range(16)] + \
    [SGDClassifier(n_iter=20, loss='modified_huber', class_weight='auto', n_jobs=1) 
     for _ in range(8)] + \
    [ExtraTreeClassifier(max_depth=6, min_samples_split=6, class_weight='auto')
     for _ in range(16)]


rasco_mix = rasco.Rasco(h,
                        feat_ratio=0.6,
                       n_estimators=104, max_iters=100,
                       verbose=True,
                      n_jobs=-1)

ord_rasco = simple.SimpleOrdinalClassifier(rasco_mix, n_jobs=1)

ord_gaussnb = simple.SimpleOrdinalClassifier(GaussianNB(), n_jobs=-1)


INFO:rasco:RASCO Init


## META MODELS

### NN META

In [13]:
n_feats = 15
drop_prob = 0.5

model_meta = Sequential()
hidden_size = 128
model_meta.add(Dense(n_feats, hidden_size))
model_meta.add(Dropout(0.2))
model_meta.add(ParametricSoftplus(hidden_size))
model_meta.add(BatchNormalization((hidden_size,)))
model_meta.add(Dropout(drop_prob))
model_meta.add(Dense(hidden_size, hidden_size))
model_meta.add(ParametricSoftplus(hidden_size))
model_meta.add(BatchNormalization((hidden_size,)))
model_meta.add(Dropout(drop_prob))
model_meta.add(Dense(hidden_size, hidden_size))
model_meta.add(ParametricSoftplus(hidden_size))
model_meta.add(BatchNormalization((hidden_size,)))
model_meta.add(Dropout(drop_prob))
model_meta.add(Dense(hidden_size, 1))
model_meta.add(Activation('linear'))

loss_type_meta = 'msle'

opt_meta = RMSprop(lr=0.0005, rho=0.75, epsilon=1e-6)

nn_meta = KerasRegressor(model=model_meta, optimizer=opt_meta, loss=loss_type_meta,
                        train_batch_size=1024*4, test_batch_size=1024*16,
                        nb_epoch=210)

# THE STACK

In [15]:
fh = logging.FileHandler('/tmp/lmg.log')

stack = stacking.Stacking([
        #            Ind   Output Dim
        nn_reg,      # 0     1
        gbm_reg,     # 1     1
        rf_reg,      # 2     1
        lin_reg,     # 4     1
        ridge_reg,   # 5     1
        ord_gaussnb, # 6     5
        ord_rasco,   # 7     5
    ],
                          nn_meta,
#                           meta_gbm,
#                           LinearRegression(),
                          fit_params={
        'base0_X': 'X_nn_train',
        'base6_y': 'y_binned',
        'base7_y': 'y_binned',
    },
                          pred_params={
        'base0_X': 'X_nn_test',
    },
                          extra_data={
        'X_nn_train': X_nn_train,
        'X_nn_test': X_nn_test,
        'y_binned': y_binned,
    },
                          include_orig_feats=False,
                          use_probs=True,
                          cv=8,
                          verbose=1,
                          save_level0_out=True,
                          log_handler=fh,
                         )

SyntaxError: invalid syntax (<ipython-input-15-9ca68bd87784>, line 27)